## Trainable Positional embeddings -- Attention is all we need

In [116]:
import numpy as np
import torch
from torch.autograd import Variable
from collections import Counter
import torch.utils.data as data_utils
from keras.datasets import imdb

### Lets load the IMDB dats

In [205]:
#Hyper params
MAXLEN = 100
BATCH_SIZE=1
EMB_DIM = 50


In [188]:
train_set,test_set = imdb.load_data(num_words=1000, index_from=3)
x_train,y_train = train_set[0],train_set[1]
x_test,y_test = test_set[0],test_set[1]
word_to_id = imdb.get_word_index()
word_to_id = {k:(v+3) for k,v in word_to_id.items()}
word_to_id["<PAD>"] = 0
word_to_id["<START>"] = 1
word_to_id["<UNK>"] = 2
x_train_pad = pad_sequences(x_train,maxlen=MAXLEN)
train_data = data_utils.TensorDataset(torch.from_numpy(x_train_pad).type(torch.LongTensor),torch.from_numpy(y_train).type(torch.DoubleTensor))
train_loader = data_utils.DataLoader(train_data,batch_size=BATCH_SIZE,drop_last=True)

### Positional Encodings .  Applies sin and cos for consecutive embedding dimensions

In [185]:
def position_encoding_init(n_position, emb_dim):
    ''' Init the sinusoid position encoding table '''

    # keep dim 0 for padding token position encoding zero vector
    position_enc = np.array([
        [pos / np.power(10000, 2 * (j // 2) / emb_dim) for j in range(emb_dim)]
        if pos != 0 else np.zeros(emb_dim) for pos in range(n_position)])
    

    position_enc[1:, 0::2] = np.sin(position_enc[1:, 0::2]) # dim 2i
    position_enc[1:, 1::2] = np.cos(position_enc[1:, 1::2]) # dim 2i+1
    return torch.from_numpy(position_enc).type(torch.FloatTensor)

In [186]:
def load_embeddings(use_pretrained_embeddings,embeddings,vocab_size,emb_dim):
    """Load the embeddings based on flag"""

    if use_pretrained_embeddings is True and embeddings is None:
        raise Exception("Send a pretrained word embedding as an argument")

    if not use_pretrained_embeddings and vocab_size is None:
        raise Exception("Vocab size cannot be empty")

    if not use_pretrained_embeddings:
        word_embeddings = torch.nn.Embedding(vocab_size,emb_dim,padding_idx=0)

    elif use_pretrained_embeddings:
        word_embeddings = torch.nn.Embedding(embeddings.size(0), embeddings.size(1),padding_idx=0)
        word_embeddings.weight = torch.nn.Parameter(embeddings)
        emb_dim = embeddings.size(1)

    return word_embeddings,emb_dim

### Loads glove Vectors

In [187]:
def load_glove_embeddings(path, word2idx, embedding_dim):
    """Loading the glove embeddings"""
    with open(path) as f:
        embeddings = np.zeros((len(word2idx), embedding_dim))
        for line in f.readlines():
            values = line.split()
            word = values[0]
            index = word2idx.get(word)
            if index:
                vector = np.array(values[1:], dtype='float32')
                if vector.shape[-1] != embedding_dim:
                    raise Exception('Dimension not matching.')
                embeddings[index] = vector
        return torch.from_numpy(embeddings).float()

### Encoding Layer (Add word embeddings to positional embeddings)

#### This layer can replace RNN/LSTM/GRU Layer . We can assume the output of this layer as hidden states of RNN. Note that you needto use attention on top of these results to make it powerful . The positional embeddings are made trainable by using embedding layer which can be set to False. 
#### Position of words can either be absolute positions as in this case or relative in nature.

In [206]:
class PositionalEncoder(torch.nn.Module):
    def __init__(self,vocab_size,emb_dim,max_len,batch_size):        
        super(PositionalEncoder,self).__init__()
        self.batch_size = batch_size
        self.emb_dim = emb_dim
        self.max_len = max_len
        n_position = max_len+1
        self.position_enc = torch.nn.Embedding(n_position, emb_dim, padding_idx=0)
        self.position_enc.weight.data = position_encoding_init(n_position, emb_dim)
        self.src_word_emb = torch.nn.Embedding(vocab_size,emb_dim,padding_idx=0)
        
    def get_absolute_pos(self,word_sequences):
        batch = []
        for word_seq in word_sequences:
            start_idx = 1
            word_pos = []
            for pos in word_seq:
                if int(pos) == 0:
                    word_pos.append(0)
                else:
                    word_pos.append(start_idx)
                    start_idx+=1
            batch.append(torch.from_numpy(np.array(word_pos)).type(torch.LongTensor))
        batch = torch.cat(batch).view(self.batch_size,self.max_len)        
        return Variable(batch)
        
    def forward(self,word_seq):
        word_embeddings = self.src_word_emb(word_seq)
        word_pos = self.get_absolute_pos(word_seq)
        word_pos_encoded = word_embeddings + self.position_enc(word_pos)
        return word_pos_encoded
        
    
        

#### Initialize the model

In [201]:
pe = PositionalEncoder(len(word_to_id),EMB_DIM,MAXLEN,BATCH_SIZE)

#### Output we get is positional encodings for each sample. I have taken care for batches as well.

In [203]:
for batch_idx,train in enumerate(train_loader):
    x,y = Variable(train[0]),Variable(train[0])
    print(encoder(x))

Variable containing:
( 0 ,.,.) = 
  0.7066  0.8641 -1.1925  ...  -0.0003 -1.1059  1.6053
  1.3403 -1.4455  2.0442  ...   1.2934 -0.7436  2.6458
 -0.3538 -0.6726  1.6542  ...   0.5997 -0.7265  0.7210
           ...             ⋱             ...          
 -1.2239  0.6689  0.3206  ...   0.1457  1.3620  2.4634
 -0.9244 -1.7156 -0.3310  ...   0.4060 -1.3912  2.4010
  0.4474  0.4733  1.6176  ...   1.4247  0.0831  1.8190
[torch.FloatTensor of size 1x100x50]

Variable containing:
( 0 ,.,.) = 
  7.8902e-01 -3.9851e-02  1.4270e+00  ...   8.8065e-02  2.3181e+00  4.2238e-01
 -3.9045e-01 -6.4186e-01  1.3030e-01  ...   1.6006e+00  3.0578e-01  1.2963e+00
  6.2489e-03 -6.6624e-01 -9.5508e-01  ...  -3.1525e-04 -1.1056e+00  1.6053e+00
                 ...                   ⋱                   ...                
 -1.7822e+00 -3.5428e-01 -1.7796e+00  ...   2.2809e+00  6.4262e-01 -9.2479e-01
 -3.5316e+00 -5.5632e-01 -8.1995e-01  ...   2.5943e+00 -8.8524e-01  1.2263e-01
 -1.6183e-01  1.3255e+00 -1.0135e+0

Variable containing:
( 0 ,.,.) = 
  1.0338e+00  9.7500e-01  1.7023e+00  ...   1.1680e+00 -1.1927e+00  2.9984e+00
 -2.9955e-01  4.8861e-02  1.7055e-01  ...   2.2811e+00  6.2874e-01 -9.2469e-01
  6.2489e-03 -6.6624e-01 -9.5508e-01  ...  -3.1525e-04 -1.1056e+00  1.6053e+00
                 ...                   ⋱                   ...                
 -5.3657e-01  9.0325e-01 -1.9008e+00  ...  -4.2841e-01 -2.9108e-01  1.0165e+00
 -8.6487e-01 -9.4575e-01 -6.7776e-01  ...   2.3765e+00  3.4493e-01  5.7608e-01
  1.1207e-01  2.0149e+00 -3.1385e-01  ...  -3.0596e-01 -3.2898e-01  6.0156e-01
[torch.FloatTensor of size 1x100x50]

Variable containing:
( 0 ,.,.) = 
  0.1220 -0.5827 -0.0992  ...   1.4424  0.9044  3.2449
  1.5277  0.7364  0.6007  ...  -0.3057 -0.3431  0.6017
  1.5192 -0.0018  2.4562  ...   1.9759  0.6802 -0.2101
           ...             ⋱             ...          
 -0.7083 -0.4955 -2.7980  ...  -0.0005 -1.0919  1.6052
 -1.1341  0.3636 -2.4145  ...  -0.0005 -1.0917  1.6052
 -0.6412  1

Variable containing:
( 0 ,.,.) = 
  7.0660e-01  8.6405e-01 -1.1925e+00  ...  -3.1507e-04 -1.1059e+00  1.6053e+00
  2.0440e-01 -3.4138e-01  1.0142e+00  ...   1.3936e+00 -8.5887e-02  8.8493e-01
  6.2489e-03 -6.6624e-01 -9.5508e-01  ...  -3.1525e-04 -1.1056e+00  1.6053e+00
                 ...                   ⋱                   ...                
 -5.2925e-01 -1.0422e+00 -1.6426e+00  ...  -1.1737e+00 -1.0337e+00  6.7686e-01
 -1.4941e+00  3.5720e-01  1.9480e-01  ...   5.9950e-01 -7.1267e-01  7.2087e-01
 -1.0795e+00 -2.3091e-01 -2.3305e-01  ...  -6.5140e-01  3.2153e-01  1.2067e+00
[torch.FloatTensor of size 1x100x50]

Variable containing:
( 0 ,.,.) = 
  1.9798e+00  9.9084e-01  2.1637e-01  ...   1.5141e+00 -3.3241e-01  3.3803e-01
  1.4245e-01 -1.5699e+00  1.6048e+00  ...   9.4808e-01 -3.1109e-01  6.1125e-01
  1.1418e+00 -1.0223e+00  1.3979e+00  ...   5.1613e-01  6.4551e-01  1.5085e+00
                 ...                   ⋱                   ...                
  5.6499e-01 -3.6875e-01 

Variable containing:
( 0 ,.,.) = 
  0.0332 -0.5414  1.7982  ...   0.5174  1.1235  0.7741
  2.0477  0.0344  0.5610  ...   1.5141 -0.3323  0.3380
 -0.7981 -1.0515  1.6639  ...   0.6732 -0.2442  1.2415
           ...             ⋱             ...          
 -0.1423 -1.8486  0.0941  ...   1.2932 -0.7297  2.6457
 -0.0455 -0.3492  0.9655  ...   1.4247  0.0829  1.8190
 -0.6412  1.1861 -1.7624  ...  -0.0005 -1.0916  1.6052
[torch.FloatTensor of size 1x100x50]

Variable containing:
( 0 ,.,.) = 
  0.7066  0.8641 -1.1925  ...  -0.0003 -1.1059  1.6053
  0.8666  0.3321  1.2771  ...   1.2800 -0.4262 -0.2646
  0.0984 -0.2417  1.1699  ...   1.2800 -0.4261 -0.2646
           ...             ⋱             ...          
 -0.5157 -0.6618 -1.4082  ...   0.3131  0.3966  3.0645
 -0.8649 -0.9457 -0.6778  ...   2.3765  0.3449  0.5761
  0.3708  2.4797 -0.6468  ...  -0.0106  0.3537 -0.8492
[torch.FloatTensor of size 1x100x50]

Variable containing:
( 0 ,.,.) = 
  0.7066  0.8641 -1.1925  ...  -0.0003 -1.1059  1.60

Variable containing:
( 0 ,.,.) = 
  1.1856  2.1764 -1.8076  ...   0.3437  0.7586  1.8399
  0.7744 -0.0924 -0.8479  ...  -0.0003 -1.1057  1.6053
  0.0062 -0.6662 -0.9551  ...  -0.0003 -1.1056  1.6053
           ...             ⋱             ...          
 -0.4445 -0.5316 -0.1751  ...   1.3250  0.4935  0.8806
 -1.2168 -0.1121 -1.6820  ...   1.4471  0.2820  0.6350
 -2.2475  0.8992  1.8783  ...   1.9003  0.4300  1.1317
[torch.FloatTensor of size 1x100x50]

Variable containing:
( 0 ,.,.) = 
  7.2476e-01  9.6668e-01  1.1076e+00  ...   4.2176e-01 -9.6079e-01  8.6634e-01
  7.0621e-01 -9.9352e-01  1.0587e+00  ...   1.8182e-01 -9.3686e-01 -1.2769e-01
 -7.1281e-01 -3.0371e+00  1.1028e-01  ...   1.3297e+00  5.3800e-01  9.8101e-01
                 ...                   ⋱                   ...                
 -3.1667e-01 -1.8149e+00 -5.5102e-01  ...   9.0997e-01  1.0337e+00  2.5081e+00
 -1.0119e+00 -6.9592e-01  1.7224e-01  ...   6.8226e-01  1.4003e+00  5.6138e-01
 -6.4124e-01  1.1861e+00 -1.7624e+0

...

Variable containing:
( 0 ,.,.) = 
  8.2906e-01  1.6082e+00  1.2579e+00  ...   2.6139e-01 -6.5458e-01  9.0670e-01
  2.0477e+00  3.4396e-02  5.6096e-01  ...   1.5141e+00 -3.3227e-01  3.3803e-01
 -1.3153e+00  4.8314e-01  1.3186e+00  ...  -2.4720e-01 -1.6090e+00  7.5365e-01
                 ...                   ⋱                   ...                
 -7.0825e-01 -4.9554e-01 -2.7980e+00  ...  -5.2470e-04 -1.0919e+00  1.6052e+00
 -8.0980e-01 -4.5360e-01  2.7746e-01  ...   1.8564e+00  5.4366e-01  1.0957e-01
 -3.1696e-01  3.6889e-01  9.2957e-01  ...   1.8564e+00  5.4380e-01  1.0957e-01
[torch.FloatTensor of size 1x100x50]

Variable containing:
( 0 ,.,.) = 
  7.0360e-01 -6.8402e-01  5.6467e-01  ...   4.3959e-01 -6.4738e-01  6.8184e-01
 -3.9045e-01 -6.4186e-01  1.3030e-01  ...   1.6006e+00  3.0578e-01  1.2963e+00
  1.0833e+00 -2.1400e+00  1.8625e-02  ...   1.1627e+00  6.7210e-01  2.0964e+00
                 ...                   ⋱                   ...                
 -1.0683e+00 -5.0191e-01 

Variable containing:
( 0 ,.,.) = 
  0.0000  0.0000  0.0000  ...   0.0000  0.0000  0.0000
  0.0000  0.0000  0.0000  ...   0.0000  0.0000  0.0000
  0.0000  0.0000  0.0000  ...   0.0000  0.0000  0.0000
           ...             ⋱             ...          
 -1.3189 -1.0417  0.6216  ...   1.6806 -0.2482  1.7086
  0.7415 -0.8274  0.7291  ...   0.8715  0.2026  0.6768
 -0.4397 -0.6287 -2.4507  ...  -0.0004 -1.0974  1.6052
[torch.FloatTensor of size 1x100x50]

Variable containing:
( 0 ,.,.) = 
  2.0943  0.2535  0.7022  ...   1.9114  0.7183  2.4682
  1.1743 -0.7105  1.2030  ...   0.2893  1.7235  4.1879
 -0.9000 -0.9307  0.1847  ...   1.7069 -0.0058  1.5119
           ...             ⋱             ...          
 -0.3084 -1.1136 -0.7472  ...   0.2891  1.7374  4.1878
 -1.8692 -0.0871 -0.2770  ...   1.2140 -0.6995  0.9155
 -0.8115  0.1540 -0.2028  ...   1.6405  0.0938  1.9684
[torch.FloatTensor of size 1x100x50]

Variable containing:
( 0 ,.,.) = 
  0.4795  1.4439  0.2249  ...   1.0898 -0.6961  1.56

Variable containing:
( 0 ,.,.) = 
 -0.8438 -0.3221 -0.2737  ...   1.7621  0.6506  0.4951
  1.8515 -1.5661  0.1258  ...   1.1627  0.6720  2.0964
 -0.1714 -3.0498  0.3679  ...   1.2850 -0.8898  0.4765
           ...             ⋱             ...          
 -0.7083 -0.4955 -2.7980  ...  -0.0005 -1.0919  1.6052
 -0.5682 -0.9895  0.4776  ...   1.2932 -0.7296  2.6457
 -1.6569  0.6981 -1.7649  ...   1.0200  0.2329  1.5214
[torch.FloatTensor of size 1x100x50]

Variable containing:
( 0 ,.,.) = 
  1.1856e+00  2.1764e+00 -1.8076e+00  ...   3.4365e-01  7.5863e-01  1.8399e+00
  1.8630e+00 -8.0516e-01  2.5321e+00  ...   1.4249e+00  6.8919e-02  1.8191e+00
  1.2795e+00 -5.3945e-01  4.5374e-01  ...   1.5141e+00 -3.3212e-01  3.3803e-01
                 ...                   ⋱                   ...                
 -1.2928e+00 -1.9422e+00 -1.7048e+00  ...   1.4422e+00  9.1843e-01  3.2448e+00
 -5.2239e-01 -1.5295e+00 -1.1713e+00  ...   1.5287e+00  1.0217e-01  1.7265e-01
 -1.9655e+00  1.8577e+00  1.1101e-0

Variable containing:
( 0 ,.,.) = 
 -0.0286  0.4134  0.9450  ...   1.2142 -0.7137  0.9156
  2.1950 -1.0497  2.1052  ...   1.7653 -0.9183  1.1209
  1.0948 -1.3790  2.4249  ...   1.4249  0.0691  1.8191
           ...             ⋱             ...          
 -1.6239 -1.2922 -1.1442  ...   2.7532 -0.5119  0.8594
 -0.8484  1.4230 -0.8081  ...  -0.2357  0.3200  2.1386
 -0.6412  1.1861 -1.7624  ...  -0.0005 -1.0916  1.6052
[torch.FloatTensor of size 1x100x50]

Variable containing:
( 0 ,.,.) = 
  0.2819  1.0249  1.7972  ...   0.6042  0.6505  1.4713
  0.8966 -1.1519  1.7389  ...   0.6825  1.3863  0.5615
  0.0244 -0.5636  1.3450  ...   0.4218 -0.9605  0.8663
           ...             ⋱             ...          
 -0.5861 -1.5550 -0.2113  ...   0.6823  1.4001  0.5614
 -1.5171  0.1053  1.4357  ...   1.4293  0.4310  1.1647
  0.8201  0.5082  2.8915  ...   0.1377  1.1786 -0.3562
[torch.FloatTensor of size 1x100x50]

Variable containing:
( 0 ,.,.) = 
  0.7066  0.8641 -1.1925  ...  -0.0003 -1.1059  1.60

KeyboardInterrupt: 